In [63]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib.image import imread

In [64]:
def make_hello(N=1000, rseed=42):
    fig, ax = plt.subplots(figsize = (4,1))
    fig.subplots_adjust(top=1, bottom=0, left=0, right=1)
    ax.axis('off')
    ax.text(0.5, 0.4, 'HELLO', va='center', ha='center', weight='bold', size=85)
    fig.savefig('hello.png')
    plt.close(fig)
    
    data = imread('hello.png')[::-1, :, 0].T
    rng = np.random.RandomState(rseed)
    X = rng.rand(4 * N, 2)
    i, j = (X * data.shape).astype(int).T
    mask = (data[i, j] < 1)
    X = X[mask]
    X[:, 0] *= (data.shape[0] / data.shape[1])
    X = X[:N]
    
    return X[np.argsort(X[:, 0])]

def random_projection(X, dimension=3, rseed=42):
    assert dimension >= X.shape[1]
    rng = np.random.RandomState(rseed)
    C = rng.randn(dimension, dimension)
    e, Y = np.linalg.eigh(np.dot(C, C.T))
    
    return np.dot(X, Y[:X.shape[1]])

### [::-1, :, 0]
  - '::-1': 이 부분은 배열의 첫 번째 차원(보통 행을 의미)에 대해 역순으로 슬라이싱
    - 즉, 배열의 마지막 요소부터 시작해 첫 번째 요소로 이동
  - ':': 두 번째 차원(보통 열을 의미)에 대해서는 전체 범위를 선택
  - '0': 세 번째 차원에서 첫 번째 요소만을 선택

In [65]:
X = make_hello(1000)

X1 = random_projection(X, 3)
X1.shape

(1000, 3)

In [66]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=X1[:, 0], y=X1[:, 1], z=X1[:, 2],
    opacity=0.3,
    mode='markers',
    marker=dict(size=5, color=X1[:, 2], colorscale='Viridis'))
)
fig.update_layout(height=800, width=800)
fig.show()

# HELLO에 대한 MDS

In [67]:
from sklearn.manifold import MDS

mds = MDS(
    n_components=2, # 3D -> 2D
    random_state=1,
    normalized_stress="auto")
out_mds = mds.fit_transform(X1)
print(out_mds[:5])

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=out_mds[:, 0], y=out_mds[:, 1],
    mode='markers',
    marker=dict(size=5, color=out_mds[:, 1], colorscale='Viridis'))
)
fig.update_layout(height=500, width=500)
fig.show()

[[-0.4497622  -1.8207143 ]
 [-0.26918837 -1.8806143 ]
 [-0.26591131 -1.87822051]
 [-0.62879339 -1.75620828]
 [-0.71833169 -1.72592393]]


# Nonlinear dimensionality reduction technique

In [68]:
def make_hello_scurve(X):
    t = (X[:, 0] - 2) * 0.75 * np.pi
    x = np.sin(t)
    y = X[:, 1]
    z = np.sign(t) * (np.cos(t) - 1)
    
    return np.vstack((x, y, z)).T

In [69]:
import plotly.graph_objects as go

XS = make_hello_scurve(X)

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=XS[:, 0], y=XS[:, 1], z=XS[:, 2],
    opacity=0.5,
    mode='markers',
    marker=dict(size=5, color=XS[:, 2], colorscale='Viridis'))
)
fig.update_layout(height=500)
fig.show()

# MDS에 의한 S자형 HELLO 축소

In [70]:
model = MDS(
    n_components=2, # 3D -> 2D
    random_state=3,
    normalized_stress="auto")
mds_out = model.fit_transform(XS)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mds_out[:, 0], y=mds_out[:, 1],
    opacity=0.5,
    mode='markers',
    marker=dict(size=3, color=mds_out[:, 1], colorscale='Viridis'))
)
fig.update_layout(height=500, width=500)
fig.show()

# ISOMAP에 의한 S자형 HELLO 축소

In [71]:
from sklearn.manifold import Isomap

iso = Isomap(
    n_neighbors=150, # vs 100
    n_components=2
)
iso_out = iso.fit_transform(XS)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iso_out[:, 0], y=iso_out[:, 1],
    mode='markers',
    marker=dict(size=5, color=iso_out[:, 0], colorscale='Viridis'))
)
fig.update_layout(height=500, width=500)
fig.show()

# LLE에 의한 S자형 HELLO 축소

In [72]:
from sklearn.manifold import LocallyLinearEmbedding as LLE

lle = LLE(
    n_neighbors=100,
    n_components=2,
    method="modified"
)
lle_out = lle.fit_transform(XS)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=lle_out[:, 0], y=lle_out[:, 1],
    mode='markers',
    marker=dict(size=5, color=lle_out[:, 0], colorscale='Viridis'))
)
fig.update_layout(height=400, width=400)
fig.show()

# TSNE에 의한 S자형 HELLO 축소

In [79]:
from sklearn.manifold import TSNE

tsne = TSNE(
    n_components=2,
    perplexity=40 # hyperparameter
)
tsne_out = tsne.fit_transform(XS)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=tsne_out[:, 0], y=tsne_out[:, 1],
    mode='markers',
    marker=dict(size=5, color=tsne_out[:, 0], colorscale='Viridis'))
)
fig.update_layout(height=400, width=400)
fig.show()